In [71]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch

from torchvision import transforms

from loader.dataset import VizwizDataset
from IPython.core.display import HTML

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# %pip install ipywidgets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
train = VizwizDataset(dtype='train', ret_type='tensor', copy_img_to_mem=False, device=device, partial=100)

loading annotations into memory...
Done (t=0.38s)
creating index...
index created! imgs = 23431, anns = 100575
tokenizing caption... done!!


In [73]:
vocabulary = train.getVocab()

In [75]:
val = VizwizDataset(dtype='val', ret_type='tensor', copy_img_to_mem=False, vocabulary=vocabulary, device=device, partial=50)

loading annotations into memory...
Done (t=0.21s)
creating index...
index created! imgs = 7750, anns = 33145
tokenizing caption... done!!


In [77]:
test = VizwizDataset(dtype='test', ret_type='tensor', copy_img_to_mem=False, vocabulary=vocabulary, device=device, partial=50)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created! imgs = 8000, anns = 0
no caption found... done!!


In [81]:
val[0]

(tensor([[[0.3294, 0.3725, 0.3490,  ..., 0.3216, 0.3451, 0.3529],
          [0.3765, 0.3686, 0.3765,  ..., 0.3255, 0.3333, 0.3451],
          [0.3608, 0.3725, 0.3686,  ..., 0.3412, 0.3490, 0.3412],
          ...,
          [0.9490, 0.9647, 0.9804,  ..., 0.9882, 0.9686, 0.9725],
          [0.9490, 0.9569, 0.9765,  ..., 0.9725, 0.9725, 0.9725],
          [0.9529, 0.9569, 0.9725,  ..., 0.9490, 0.9608, 0.9529]],
 
         [[0.4314, 0.4745, 0.4510,  ..., 0.3412, 0.3608, 0.3686],
          [0.4706, 0.4706, 0.4784,  ..., 0.3451, 0.3569, 0.3608],
          [0.4549, 0.4667, 0.4706,  ..., 0.3725, 0.3686, 0.3647],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.9765, 0.9608, 0.9569],
          [1.0000, 1.0000, 0.9961,  ..., 0.9608, 0.9569, 0.9569],
          [1.0000, 1.0000, 1.0000,  ..., 0.9373, 0.9451, 0.9373]],
 
         [[0.6157, 0.6588, 0.6353,  ..., 0.4627, 0.4745, 0.4824],
          [0.6510, 0.6510, 0.6627,  ..., 0.4667, 0.4706, 0.4745],
          [0.6353, 0.6471, 0.6510,  ...,

In [41]:
from loader import vizwiz as vw

vizwiz = vw.VizWiz(annotation_file='annotations/val.json')
df = pd.DataFrame.from_dict(vizwiz.dataset['images'], orient='columns')

loading annotations into memory...
Done (t=0.11s)
creating index...
index created! imgs = 7750, anns = 33145


In [67]:
s = df.iloc[0]
all([i in s for i in ['row', 'file_name']])

False

In [53]:
s

file_name                                  VizWiz_val_00000000.jpg
vizwiz_url       https://ivc.ischool.utexas.edu/VizWiz_visualiz...
id                                                           23431
text_detected                                                 True
Name: 0, dtype: object